## PYTHON CIRCUIT SOLVER

In [1]:
%reload_ext Cython

In [2]:
%%cython --annotate
import numpy as np
import time

filename = "ckt3.netlist"
start_time = time.time()
def solver1(A,b):
    try:
        A,b=interchange(A,b)
        matrix=A
        n = len(A)
        m = len(A[0])
        for i in range(n):
            matrix[i].append(b[i])
        # print(np.array(matrix))
        for i in range(n):
            matrix[i] = [matrix[i][j] / matrix[i][i] for j in range(m + 1)]
            for j in range(i + 1, n):
                factor = matrix[j][i] / matrix[i][i]
                matrix[j] = [matrix[j][k] - factor * matrix[i][k] for k in range(m + 1)]
        x = [0 for _ in range(n)]
        # print(np.array(matrix))
        for i in range(n - 1, -1, -1):
            x[i] = (matrix[i][m] - sum(x[j] * matrix[i][j] for j in range(i + 1, n))) / matrix[i][i]
            # print(matrix[i][i])
            # print(x)
        return x
        
    except :
        return "No Solution or Infinite Solutions"

def interchange(A,b):
    n=len(A[0])
    for i in range (n):
        if A[i][i] ==0:
            s=0
            for j in range(i+1,n):
                if s==1:
                    break
                if A[j][j]!= 0:
                    for k in range(n):
                        temp=A[i][k]
                        A[i][k]=A[j][k]
                        A[j][k]= temp
                    temp=b[i]
                    b[i]=b[j]
                    b[j]=temp
                    s=1
                    break
                    
    return A,b

def solver(A, b):
    det = np.linalg.det(A)
    n=len(b)
    if det == 0:
        return "No solution or Infinite Solutions"
    else:
        return np.linalg.solve(A, b)
    

with open(filename) as f:
    lines = f.readlines()
    c = 0
    e = 0
    volt=0
    w=0
    nodes = set()
    resistor = []
    inductor = []
    capacitor = []
    voltage = []
    current = []
    for line in lines:
        s=line.split()
        if s[0] == ".circuit":
            c = 1
            continue
        if s[0]==".ac":
            w=float(s[2])
        elif s[0] == ".end":
            e = 1
            continue
        if c==1 and e==1:
            break
        if c == 1 and e == 0:
            l=line.split()
            if l[1] == "GND":
                node1 = int(l[2])
                node2 = "GND"
            elif l[2] == "GND":
                node1 = int(l[1])
                node2 = "GND"
            else:
                node1= int(l[1])
                node2=int(l[2])
                nodes.add(node1)
                nodes.add(node2)
            nodes.add(node1)
            element_type = l[0][0]
            if element_type == 'V':
                volt+=1
                voltage.append([l[1], l[2], l[3], l[4]])
            elif element_type == 'I':
                current.append([l[1], l[2], l[3], l[4]])
            elif element_type == 'R':
                a=[l[1], l[2], l[3]]
                resistor.append(a)
            elif element_type == 'L':
                a=[l[1], l[2], l[3]]
                inductor.append(a)
            elif element_type == 'C':
                a=[l[1], l[2], l[3]]
                capacitor.append(a)

    n=max(nodes)+volt
    A=[[complex(0) for _ in range(n)]for _ in range(n)]
    b=[complex(0) for _ in range(n)]

    for node1, node2, r in resistor:
        r=float(r)
        if node1=="GND":
            node2=int(node2)
            node2 -= 1
            A[node2][node2] += 1/r
        elif node2=="GND":
            node1=int(node1)
            node1 -= 1
            A[node1][node1] += 1/r
        else:

            node1=int(node1)
            node2=int(node2)
            node1 -= 1
            node2 -= 1
            A[node1][node1] += 1/r
            A[node2][node2] += 1/r
            A[node1][node2] -= 1/r
            A[node2][node1] -= 1/r

    for node1, node2, c in capacitor:
        c=float(c)
        if node1=="GND":
            node2=int(node2)
            node2 -= 1
            A[node2][node2] += (w*c)*1j
        elif node2=="GND":
            node1=int(node1)
            node1 -= 1
            A[node1][node1] += (w*c)*1j
        else:

            node1=int(node1)
            node2=int(node2)
            node1 -= 1
            node2 -= 1
            A[node1][node1] += (w*c)*1j
            A[node2][node2] += w*c*1j
            A[node1][node2] -= (w*c)*1j
            A[node2][node1] -= (w*c)*1j
    
    for node1, node2, l in inductor:
        l=float(l)
        if node1=="GND":
            node2=int(node2)
            node2 -= 1
            A[node2][node2] +=1/( w*l*1j)
        elif node2=="GND":
            node1=int(node1)
            node1 -= 1
            A[node1][node1] +=1/ ( w*l*1j)
        else:

            node1=int(node1)
            node2=int(node2)
            node1 -= 1
            node2 -= 1
            A[node1][node1] +=1/ ( w*l*1j)
            A[node2][node2] +=1/ ( w*l*1j)
            A[node1][node2] -=1/ ( w*l*1j)
            A[node2][node1] -=1/ ( w*l*1j)

    
    for node1,node2,typ,c in current:
        c=float(c)
        if True:
            if node1=="GND":
                node2=int(node2)
                node2 -= 1
                b[node2]+=c
            elif node2=="GND":
                node1=int(node1)
                node1 -= 1
                b[node1]-=c
            else:
                node2=int(node2)
                node2 -= 1
                b[node2]+=c
                node1=int(node1)
                node1 -= 1
                b[node1]-=c

    for node1,node2,typ,v in voltage:
        v=float(v)
        if True:
            if node1=="GND":
                node2=int(node2)
                node2 -= 1
                A[node2][n-1]+=1
                A[n-1][node2]+=1
                b[n-1]+=v
            elif node2=="GND":
                node1=int(node1)
                node1 -= 1
                A[node1][n-1]-=1
                A[n-1][node1]-=1
                b[n-1]-=v
            else:
                node2=int(node2)
                node2 -= 1
                A[node2][n-1]-=1
                A[n-1][node2]-=1
                b[n-1]-=v
                node1=int(node1)
                node1 -= 1
                A[node1][n-1]+=1
                A[n-1][node1]+=1
                b[n-1]+=v
            n-=1
            
A1=np.array(A)
b1=np.array(b)
print(A1,b1)
result=solver1(A,b)
print(result)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

[[ 0.001 +0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j  1.    +0.j]
 [-0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j -0.001 +0.j  0.0015+0.j  0.    +0.j]
 [ 1.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]] [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j 10.+0.j]
[(10+0j), (5.029239766081871+0j), (2.573099415204678+0j), (1.4035087719298247+0j), (0.9356725146198832+0j), (-0.004970760233918129-0j)]
Elapsed time: 0.0014605522155761719 seconds


In [3]:
%timeit solver1(A,b)

751 ns ± 11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## IMPROVED PYTHON CIRCUIT SOLVER

In [4]:
%%cython --annotate

import numpy as np
import time
import cython

filename = "ckt3.netlist"
start_time = time.time()

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def solver1(A,b):
    cdef int n,m,i,j
    try:
        A,b=interchange(A,b)
        matrix=A
        n = len(A)
        m = len(A[0])
        for i in range(n):
            matrix[i].append(b[i])
        for i in range(n):
            matrix[i] = [matrix[i][j] / matrix[i][i] for j in range(m + 1)]
            for j in range(i + 1, n):
                factor = matrix[j][i] / matrix[i][i]
                matrix[j] = [matrix[j][k] - factor * matrix[i][k] for k in range(m + 1)]
        x = [complex(0) for _ in range(n)]
        for i in range(n - 1, -1, -1):
            x[i] = (matrix[i][m] - sum(x[j] * matrix[i][j] for j in range(i + 1, n))) / matrix[i][i]
        return x
        
    except :
        return "No Solution or Infinite Solutions"

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)    
def interchange(A,b):
    cdef int n,i,j,k,s
    cdef complex temp
    n=len(A[0])
    for i in range (n):
        if A[i][i] ==0:
            s=0
            for j in range(i+1,n):
                if s==1:
                    break
                if A[j][j]!= 0:
                    for k in range(n):
                        temp=A[i][k]
                        A[i][k]=A[j][k]
                        A[j][k]= temp
                    temp=b[i]
                    b[i]=b[j]
                    b[j]=temp
                    s=1
                    break
                    
    return A,b

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def matrix_builder(filename):
    cdef int e,volt,n
    cdef float w,C
    with open(filename) as f:
        lines = f.readlines()
        c = 0
        e = 0
        volt=0
        w=0
        nodes = set()
        resistor = []
        inductor = []
        capacitor = []
        voltage = []
        current = []
        for line in lines:
            s=line.split()
            if s[0] == ".circuit":
                c = 1
                continue
            if s[0]==".ac":
                w=float(s[2])
            elif s[0] == ".end":
                e = 1
                continue
            if c==1 and e==1:
                break
            if c == 1 and e == 0:
                l=line.split()
                if l[1] == "GND":
                    node1 = int(l[2])
                    node2 = "GND"
                elif l[2] == "GND":
                    node1 = int(l[1])
                    node2 = "GND"
                else:
                    node1= int(l[1])
                    node2=int(l[2])
                    nodes.add(node1)
                    nodes.add(node2)
                nodes.add(node1)
                element_type = l[0][0]
                if element_type == 'V':
                    volt+=1
                    voltage.append([l[1], l[2], l[3], l[4]])
                elif element_type == 'I':
                    current.append([l[1], l[2], l[3], l[4]])
                elif element_type == 'R':
                    a=[l[1], l[2], l[3]]
                    resistor.append(a)
                elif element_type == 'L':
                    a=[l[1], l[2], l[3]]
                    inductor.append(a)
                elif element_type == 'C':
                    a=[l[1], l[2], l[3]]
                    capacitor.append(a)

        n=max(nodes)+volt
        A=[[complex(0) for _ in range(n)]for _ in range(n)]
        b=[complex(0) for _ in range(n)]

        for node1, node2, r in resistor:
            r=float(r)
            if node1=="GND":
                node2=int(node2)
                node2 -= 1
                A[node2][node2] += 1/r
            elif node2=="GND":
                node1=int(node1)
                node1 -= 1
                A[node1][node1] += 1/r
            else:

                node1=int(node1)
                node2=int(node2)
                node1 -= 1
                node2 -= 1
                A[node1][node1] += 1/r
                A[node2][node2] += 1/r
                A[node1][node2] -= 1/r
                A[node2][node1] -= 1/r

        for node1, node2, c in capacitor:
            c=float(c)
            if node1=="GND":
                node2=int(node2)
                node2 -= 1
                A[node2][node2] += (w*c)*1j
            elif node2=="GND":
                node1=int(node1)
                node1 -= 1
                A[node1][node1] += (w*c)*1j
            else:

                node1=int(node1)
                node2=int(node2)
                node1 -= 1
                node2 -= 1
                A[node1][node1] += (w*c)*1j
                A[node2][node2] += w*c*1j
                A[node1][node2] -= (w*c)*1j
                A[node2][node1] -= (w*c)*1j

        for node1, node2, l in inductor:
            l=float(l)
            if node1=="GND":
                node2=int(node2)
                node2 -= 1
                A[node2][node2] +=1/( w*l*1j)
            elif node2=="GND":
                node1=int(node1)
                node1 -= 1
                A[node1][node1] +=1/ ( w*l*1j)
            else:

                node1=int(node1)
                node2=int(node2)
                node1 -= 1
                node2 -= 1
                A[node1][node1] +=1/ ( w*l*1j)
                A[node2][node2] +=1/ ( w*l*1j)
                A[node1][node2] -=1/ ( w*l*1j)
                A[node2][node1] -=1/ ( w*l*1j)


        for node1,node2,typ,c in current:
            c=float(c)
            if True:
                if node1=="GND":
                    node2=int(node2)
                    node2 -= 1
                    b[node2]+=c
                elif node2=="GND":
                    node1=int(node1)
                    node1 -= 1
                    b[node1]-=c
                else:
                    node2=int(node2)
                    node2 -= 1
                    b[node2]+=c
                    node1=int(node1)
                    node1 -= 1
                    b[node1]-=c

        for node1,node2,typ,v in voltage:
            v=float(v)
            if True:
                if node1=="GND":
                    node2=int(node2)
                    node2 -= 1
                    A[node2][n-1]+=1
                    A[n-1][node2]+=1
                    b[n-1]+=v
                elif node2=="GND":
                    node1=int(node1)
                    node1 -= 1
                    A[node1][n-1]-=1
                    A[n-1][node1]-=1
                    b[n-1]-=v
                else:
                    node2=int(node2)
                    node2 -= 1
                    A[node2][n-1]-=1
                    A[n-1][node2]-=1
                    b[n-1]-=v
                    node1=int(node1)
                    node1 -= 1
                    A[node1][n-1]+=1
                    A[n-1][node1]+=1
                    b[n-1]+=v
                n-=1
    return A,b

A,b=matrix_builder(filename)
A1=np.array(A)
b1=np.array(b)
print(A1,b1)
result=solver1(A,b)
print(result)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

[[ 0.001 +0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j  1.    +0.j]
 [-0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j -0.001 +0.j  0.0025+0.j -0.001 +0.j  0.    +0.j]
 [ 0.    +0.j  0.    +0.j  0.    +0.j -0.001 +0.j  0.0015+0.j  0.    +0.j]
 [ 1.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j  0.    +0.j]] [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j 10.+0.j]
[(10+0j), (5.029239766081871+0j), (2.573099415204678+0j), (1.4035087719298247+0j), (0.9356725146198832+0j), (-0.004970760233918129-0j)]
Elapsed time: 0.0013170242309570312 seconds


In [5]:
%timeit solver1(A,b)

564 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## EXPLAINATION

Cython allows for static type declarations, which can help the interpreter generate more efficient code. In the code, the types of all variables and function arguments are explicitly declared using the "cdef" keyword. This allows the Cython compiler to generate C code that is optimized for the declared types.
Cython allows for fine-grained control over memory allocation and deallocation. In the code, memory is allocated only once for the matrices and vectors, and no additional memory is allocated or deallocated during the execution of the code. This can improve performance and reduce memory usage. 
Decorators like @cython.cdivision(True) tells Cython to enable C-style division during compilation. When C-style division is enabled, Cython uses C rules for division, which means that dividing two integers results in a truncated integer quotient, and not a float. The "@cython.wraparound(False)" decorator tells Cython to disable negative indexing and index range checking for all arrays and sequences in the function or module it is applied to. This means that Cython will assume that indices are always within the bounds of the array or sequence, and will not perform any checks to ensure that an index is within the range of valid indices. This can improve the performance of array and sequence indexing operations, but can also make the code less safe if the assumption of valid indices is violated. the "@cython.boundscheck(False)"  Disables negative index check: By default, Cython checks that all indices are non-negative when accessing an array. This is done to prevent buffer overflow and improve safety. By setting @cython.boundscheck(False), you tell Cython to skip this check and allow negative indices. This can improve performance but can also lead to hard-to-debug errors if you don't handle it properly.